In [2]:
#Import libraries
import os
import sys
import gc
import arcpy
from arcpy import env
from arcpy.sa import *
import time
import traceback
import tempfile
import math
import subprocess
import shutil

#These functions were taken from the ArcGIS Geomorphometry and Gradient Metrics Toolbox
#https://evansmurphy.wixsite.com/evansspatial/arcgis-gradient-metrics-toolbox
def deleteTempFiles(hdr):
    dataList = arcpy.ListDatasets(hdr+"_*")
    for d in dataList:
        try:
            arcpy.Delete_management(d)
        except:
            arcpy.AddWarning("Could not delte "+d)
            continue
    fcList = arcpy.ListFeatureClasses(hdr+"_*")
    for fc in fcList:
        try:
            arcpy.Delete_management(fc)
        except:
            arcpy.AddWarning("Could not delte "+fc)
            continue

    tableList = arcpy.ListTables(hdr+"_*")
    for t in tableList:
        try:
            arcpy.Delete_management(t)
        except:
            arcpy.AddWarning("Could not delte "+t)
            continue


def checkExt(inDem):
    #arcpy.AddWarning("here = "+env.extent)
    if (env.extent):
        outRaster = Times(inDem,1)
        return outRaster
    else:
        outRaster = inDem
        if isArcMap():
            mapSRef = returnCurrentSRefOfMap()
            desc = arcpy.Describe(inDem)
            if mapSRef.name != desc.spatialReference.name:
                env.outputCoordinateSystem = mapSRef
                outRaster = Times(inDem,1)
        return outRaster





def getZFactor(dem,inZUnits):
    #inDem =checkExt(dem)
    descR = arcpy.Describe(dem)
    inSpRef = descR.spatialReference
    spRefType = inSpRef.type

    if spRefType == "Geographic":
        #arcpy.AddWarning("Data using a Geographic Spatial Reference.  For more accurate results project your DEM using the appropriate projection with matching XY and Z units and re-run tool")
        medianLat = getMidLat(descR.extent)
        rLat = math.radians(medianLat)
        meters2Degree = 111412.84 * math.cos(rLat) - 93.5*math.cos(3*rLat)
        if inZUnits == "Meter":
        #using 1 deg long is 111.412 kms at equator
            numerator = meters2Degree
        else:
        #Using 1 deg long is 69.172 miles at equator
            numerator = meters2Degree*3.28084
        zFactor = 1/numerator
    else:
        inUnits = inSpRef.linearUnitName
        if inUnits != inZUnits:
            if inUnits == "Meter":
                #Since they don't equal this means xyUnits == Meter and zUnits == Feet
                zFactor = 0.3048
            else:
                #Means xyUnits == Feet and  zUnits == Meters (not common)
                zFactor = 3.28084
        else:
            zFactor = 1

    return zFactor

def getMidLat (rExt):
    spRefType = rExt.spatialReference.type
    if spRefType == "Geographic":
        rYMax = rExt.YMax
        rYMin = rExt.YMin
        medianLat = abs((float(rYMax)- float(rYMin))/2+rYMin)
    else:
        wgs84 = arcpy.SpatialReference(4326)
        prjExt = rExt.projectAs(wgs84)
        rYMax = prjExt.YMax
        rYMin = prjExt.YMin
        medianLat = abs((float(rYMax)- float(rYMin))/2+rYMin)
    return medianLat

def isArcMap():
	try:
		mxd = arcpy.mapping.MapDocument("CURRENT")
		return True
	except:
		return False

def returnCurrentSRefOfMap():
    mxd = arcpy.mapping.MapDocument("CURRENT")
    sRef = mxd.activeDataFrame.spatialReference
    return sRef


#Link to DEM data
dem = "D:/v_r_landslide/inputs/dem_2m.tif"
#Set input DEM as snap raster
env.snapRaster = "D:/v_r_landslide/inputs/dem_2m.tif"

#Set environment settings. 
env.overwriteOutput = True
arcpy.env.pyramids = "NONE"
arcpy.env.rasterStatistics = "NONE"
arcpy.env.compression = "NONE"
env.snapRaster = "D:/v_r_landslide/inputs/dem_2m.tif"
arcpy.env.workspace = "D:/v_r_landslide/temp"
arcpy.env.scratchWorkspace = "D:/v_r_landslide/scratch"

#Optain input tile extent as a parameter from the subprocess container
t1 = str(sys.argv[1])
#Link to tile on local disk
area = "D:/v_r_landslide/shp_out/" + t1

#Buffer tile by 100 meters
arcpy.Buffer_analysis(area, "buffer_out.shp",  "100 Meter")
area_b = "buffer_out.shp"

#Set buffered tile as the processing extent and mask
env.extent = area_b
env.mask = area_b

#Extract DEM within tile
dem_ext = ExtractByMask(dem, area_b)
dem_ext.save("D:/v_r_landslide/dem_clip/dem_ext.tif")

#Calcualte slope from DEM
slp = Slope(dem_ext, "DEGREE")
slp.save("D:/v_r_landslide/out_grids/slp.tif")

#Calculate slope position at different scales
mnEle = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MEAN")
sp21 = dem_ext - mnEle
mnEle = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MEAN")
sp11 = dem_ext - mnEle
mnEle = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MEAN")
sp7 = dem_ext - mnEle
sp21.save("D:/v_r_landslide/out_grids/sp21.tif")
sp11.save("D:/v_r_landslide/out_grids/sp11.tif")
sp7.save("D:/v_r_landslide/out_grids/sp7.tif")

#Calculate topogrpahic roughness at different scales
stdEle = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"STD")
rph21 = Square(stdEle)
stdEle = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"STD")
rph7 = Square(stdEle)
stdEle = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"STD")
rph11 = Square(stdEle)
rph21.save("D:/v_r_landslide/out_grids/rph21.tif")
rph11.save("D:/v_r_landslide/out_grids/rph11.tif")
rph7.save("D:/v_r_landslide/out_grids/rph7.tif")

#Calculate topographic dissetion at different scales
maxEle = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MAXIMUM")
minEle = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MINIMUM")
rngEle = Float(maxEle - minEle)
diss_pre = Float(dem_ext - minEle) / rngEle
diss21 = Con(rngEle==0,0,diss_pre)

#Calculate topograhic dissection at different scales
maxEle = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MAXIMUM")
minEle = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MINIMUM")
rngEle = Float(maxEle - minEle)
diss_pre = Float(dem_ext - minEle) / rngEle
diss7 = Con(rngEle==0,0,diss_pre)

maxEle = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MAXIMUM")
minEle = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MINIMUM")
rngEle = Float(maxEle - minEle)
diss_pre = Float(dem_ext - minEle) / rngEle
diss11 = Con(rngEle==0,0,diss_pre)

diss21.save("D:/v_r_landslide/out_grids/diss21.tif")
diss11.save("D:/v_r_landslide/out_grids/diss11.tif")
diss7.save("D:/v_r_landslide/out_grids/diss7.tif")

#Calculate mean slope at different scales
slpmn21 = FocalStatistics(slp,NbrCircle(21, "CELL"),"MEAN")
slpmn7 = FocalStatistics(slp,NbrCircle(7, "CELL"),"MEAN")
slpmn11 = FocalStatistics(slp,NbrCircle(11, "CELL"),"MEAN")

slpmn21.save("D:/v_r_landslide/out_grids/slpmn21.tif")
slpmn7.save("D:/v_r_landslide/out_grids/slpmn7.tif")
slpmn11.save("D:/v_r_landslide/out_grids/slpmn11.tif")

#Calcualte surface exposure index from topographic aspect
aspect = Aspect(dem_ext)
cosAsp = Cos(Divide(Times(3.142,Minus(aspect,180)),180))
sei = Times(slp,cosAsp)
sei.save("D:/v_r_landslide/out_grids/sei.tif")

#Calculate heat load index
dscRaster = arcpy.Describe(dem_ext)
ext = dscRaster.extent
midLat = getMidLat(ext)
l = float(midLat) * 0.017453293
cl = math.cos(float(l))
sl = math.sin(l)
tmp1 = slp              
tmp2 = Aspect(dem_ext) * 0.017453293              
tmp3 = Abs(3.141593 - Abs(tmp2 - 3.926991))     
tmp4 = Cos(tmp1)
tmp5 = Sin(tmp1)
tmp6 = Cos(tmp3)
tmp7 = Sin(tmp3)
hli = Exp( -1.467 +  1.582 * cl * tmp4  - 1.5 * tmp6 * tmp5 * sl - 0.262 * sl * tmp5  + 0.607 * tmp7 * tmp5)
hli.save("D:/v_r_landslide/out_grids/hli.tif")

Calculate a linear transformation of aspect
tmp1=Aspect(dem_ext)
tmp2=SetNull(tmp1<0,(450.0-tmp1)/57.296)
tmp3=Sin(tmp2)
tmp4=Cos(tmp2)
tmp5=FocalStatistics(tmp3,NbrRectangle(3,3,"CELL"),"SUM","DATA")
tmp6=FocalStatistics(tmp4,NbrRectangle(3,3,"CELL"),"SUM","DATA")
#The *100 and 36000(360*100) / 100 allow for two decimal points since Fmod appears to be gone
tmpMod = Mod(((450-(ATan2(tmp5,tmp6)*57.296))*100),36000)/100
asp_lin = Con((tmp5==0) & (tmp6==0),-1, tmpMod)
asp_lin.save("D:/v_r_landslide/out_grids/asp_lin.tif")

Calculate surface relief ratio at different scales
tmp1 = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MINIMUM")
tmp2 = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MAXIMUM")
tmp3 = FocalStatistics(dem_ext, NbrCircle(21, "CELL"),"MEAN")
conTmp = Float(tmp2 - tmp1)
outVal = Float(tmp3 - tmp1) / conTmp
ssr21 = Con(conTmp==0,0,outVal)

tmp1 = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MINIMUM")
tmp2 = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MAXIMUM")
tmp3 = FocalStatistics(dem_ext, NbrCircle(7, "CELL"),"MEAN")
conTmp = Float(tmp2 - tmp1)
outVal = Float(tmp3 - tmp1) / conTmp
ssr7 = Con(conTmp==0,0,outVal)

tmp1 = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MINIMUM")
tmp2 = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MAXIMUM")
tmp3 = FocalStatistics(dem_ext, NbrCircle(11, "CELL"),"MEAN")
conTmp = Float(tmp2 - tmp1)
outVal = Float(tmp3 - tmp1) / conTmp
ssr11 = Con(conTmp==0,0,outVal)

#Calculate surface area ratio
r = checkExt(dem_ext)
dscRaster = arcpy.Describe(r)
cellSize = dscRaster.meanCellHeight
c = cellSize * cellSize
v = math.pi/180
tmp1 = Slope(r,"DEGREE") * v
sar =  Float(c) / Cos(tmp1)

ssr21.save("D:/v_r_landslide/out_grids/ssr21.tif")
ssr11.save("D:/v_r_landslide/out_grids/ssr11.tif")
ssr7.save("D:/v_r_landslide/out_grids/ssr7.tif")
sar.save("D:/v_r_landslide/out_grids/sar.tif")

#Call to R and SAGA to calculate surface curvature measures
subprocess.call(["C:\\Program Files\\R\\R-3.5.2\\bin\\Rscript.exe", "D:/v_r_landslide/inputs/rsaga_code.R"])

#Call in surface curvature results
crossc21a = "D:/v_r_landslide/temp/SAGA_cross21.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(crossc21a, sr)
crossc21 = Times(crossc21a,1.00) 

planc21a = "D:/v_r_landslide/temp/SAGA_plan21.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(planc21a, sr)
planc21 = Times(planc21a, 1.00)

proc21a = "D:/v_r_landslide/temp/SAGA_pro21.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(proc21a, sr)
proc21 = Times(proc21a, 1.00)

longc21a = "D:/v_r_landslide/temp/SAGA_long21.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(longc21a, sr)
longc21 = Times(longc21a, 1.00)

crossc21.save("D:/v_r_landslide/out_grids/crossc21.tif")
planc21.save("D:/v_r_landslide/out_grids/planc21.tif")
proc21.save("D:/v_r_landslide/out_grids/proc21.tif")
longc21.save("D:/v_r_landslide/out_grids/longc21.tif")

crossc11a = "D:/v_r_landslide/temp/SAGA_cross11.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(crossc11a, sr)
crossc11 = Times(crossc11a, 1.00) 

planc11a = "D:/v_r_landslide/temp/SAGA_plan11.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(planc21a, sr)
planc11 = Times(planc11a, 1.00)

proc11a = "D:/v_r_landslide/temp/SAGA_pro11.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(proc11a, sr)
proc11 = Times(proc11a, 1.00)

longc11a = "D:/v_r_landslide/temp/SAGA_long11.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(longc11a, sr)
longc11 = Times(longc11a, 1.00)

crossc11.save("D:/v_r_landslide/out_grids/crossc11.tif")
planc11.save("D:/v_r_landslide/out_grids/planc11.tif")
proc11.save("D:/v_r_landslide/out_grids/proc11.tif")
longc11.save("D:/v_r_landslide/out_grids/longc11.tif")

crossc7a = "D:/v_r_landslide/temp/SAGA_cross7.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(crossc7a, sr)
crossc7 = Times(crossc7a, 1.00)

planc7a = "D:/v_r_landslide/temp/SAGA_plan7.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(planc7a, sr)
planc7 = Times(planc7a, 1.00) 

proc7a = "D:/v_r_landslide/temp/SAGA_pro7.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(proc7a, sr)
proc7 = Times(proc7a, 1.00)

longc7a = "D:/v_r_landslide/temp/SAGA_long7.sdat"
sr = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")
arcpy.DefineProjection_management(longc7a, sr)
longc7 = Times(longc7a, 1.00)

crossc7.save("D:/v_r_landslide/out_grids/crossc7.tif")
planc7.save("D:/v_r_landslide/out_grids/planc7.tif")
proc7.save("D:/v_r_landslide/out_grids/proc7.tif")
longc7.save("D:/v_r_landslide/out_grids/longc7.tif")

#Link to rock type, dominant soil parent material and drainage class grids
rock_type = "D:/v_r_landslide/inputs/tock_type.tif"
steve_comm = "D:/v_r_landslide/inputs/stv_com.tif"
dspm_class = "D:/v_r_landslide/inputs/dspm_cls.tif"
drainage_class = "D:/v_r_landslide/inputs/dspm_jim_cls.tif"

#Extract grid cells witin tile extent
rktyp = ExtractByMask(rock_type, area_b)
steve = ExtractByMask(steve_comm, area_b)
dspm = ExtractByMask(dspm_class, area_b)
drainage = ExtractByMask(drainage_class, area_b)

rktyp.save("D:/v_r_landslide/out_grids/rktyp.tif")
steve.save("D:/v_r_landslide/out_grids/steve.tif")
dspm.save("D:/v_r_landslide/out_grids/dspm.tif")
drainage.save("D:/v_r_landslide/out_grids/drainage.tif")

#Read in distance and cost distance grids
us_dist_pre = ExtractByMask("D:/v_r_landslide/inputs/us_rd_dist.tif", area_b)
us_cost_pre = ExtractByMask("D:/v_r_landslide/inputs/us_rd_cost.tif", area_b)
state_dist_pre = ExtractByMask("D:/v_r_landslide/inputs/sate_road_dist.tif", area_b)
state_cost_pre = ExtractByMask("D:/v_r_landslide/inputs/state_rd_cost.tif", area_b)
local_dist_pre = ExtractByMask("D:/v_r_landslide/inputs/local_rds_dist.tif", area_b)
local_cost_pre = ExtractByMask("D:/v_r_landslide/inputs/local_rd_cost.tif", area_b)
stream_dist_pre = ExtractByMask("D:/v_r_landslide/inputs/strms_dist.tif", area_b)
stream_cost_pre = ExtractByMask("D:/v_r_landslide/inputs/strm_cost.tif", area_b)

#Resample cross distance grids to match DEM-derived inputs
arcpy.Resample_management(us_dist_pre, "D:/v_r_landslide/out_grids/us_dist_2.tif", "2", "NEAREST")
arcpy.Resample_management(us_cost_pre, "D:/v_r_landslide/out_grids/us_cost_2.tif", "2", "NEAREST")
arcpy.Resample_management(state_dist_pre, "D:/v_r_landslide/out_grids/state_dist_2.tif", "2", "NEAREST")
arcpy.Resample_management(state_cost_pre, "D:/v_r_landslide/out_grids/state_cost_2.tif", "2", "NEAREST")
arcpy.Resample_management(local_dist_pre, "D:/v_r_landslide/out_grids/local_dist_2.tif", "2", "NEAREST")
arcpy.Resample_management(local_cost_pre, "D:/v_r_landslide/out_grids/local_cost_2.tif", "2", "NEAREST")
arcpy.Resample_management(stream_dist_pre, "D:/v_r_landslide/out_grids/stream_dist_2.tif", "2", "NEAREST")
arcpy.Resample_management( stream_cost_pre, "D:/v_r_landslide/out_grids/stream_cost_2.tif", "2", "NEAREST")

#Stack all variables into a multiband raster stack
arcpy.env.workspace = "D:/v_r_landslide/out_grids"
raster_list = arcpy.ListRasters()
pred_rasters = ["D:/v_r_landslide/out_grids/" + ras for ras in raster_list]
arcpy.CompositeBands_management(pred_rasters,"D:/v_r_landslide/stack/preds.tif")

#Predict to raster stack using random forest model in R
subprocess.call(["C:\\Program Files\\R\\R-3.5.2\\bin\\Rscript.exe", "D:/v_r_landslide/inputs/V_R_pred_script.R"])


['D:/v_r_landslide/out_grids/asp_lin.tif', 'D:/v_r_landslide/out_grids/crossc11.tif', 'D:/v_r_landslide/out_grids/crossc21.tif', 'D:/v_r_landslide/out_grids/crossc7.tif', 'D:/v_r_landslide/out_grids/diss11.tif', 'D:/v_r_landslide/out_grids/diss21.tif', 'D:/v_r_landslide/out_grids/diss7.tif', 'D:/v_r_landslide/out_grids/drainage.tif', 'D:/v_r_landslide/out_grids/dspm.tif', 'D:/v_r_landslide/out_grids/hli.tif', 'D:/v_r_landslide/out_grids/local_cost_2.tif', 'D:/v_r_landslide/out_grids/local_dist_2.tif', 'D:/v_r_landslide/out_grids/longc11.tif', 'D:/v_r_landslide/out_grids/longc21.tif', 'D:/v_r_landslide/out_grids/longc7.tif', 'D:/v_r_landslide/out_grids/planc11.tif', 'D:/v_r_landslide/out_grids/planc21.tif', 'D:/v_r_landslide/out_grids/planc7.tif', 'D:/v_r_landslide/out_grids/proc11.tif', 'D:/v_r_landslide/out_grids/proc21.tif', 'D:/v_r_landslide/out_grids/proc7.tif', 'D:/v_r_landslide/out_grids/rktyp.tif', 'D:/v_r_landslide/out_grids/rph11.tif', 'D:/v_r_landslide/out_grids/rph21.tif', '

0